# Segmenting and Clustering Neighborhoods in Toronto

## Capstone Week 3 Assignment: Applied Data Science Capstone - Part 2

### Luciano Guerra Domínguez  

## 1.- Scrape Wikipedia and customize a Dataframe with Pandas

In [7]:
# Beautiful Soup
from bs4 import BeautifulSoup
import requests

# import Pandas
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Getting the webpage, creating a Response object.
response = requests.get(url)
 
# Extracting the source code of the page.
data = response.text
 
# Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
soup = BeautifulSoup(data, 'lxml')

# Find the Table
My_table = soup.find('table',{'class':'wikitable sortable'})

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# extract information from My_table
rows_list = []
for tr in My_table.findAll('tr'):
    tds = tr.findAll('td')
    if not tds:
        continue
    postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
    rows_list.append([postcode, borough, neighborhood])

# create dataframe (without cleaning)
postalcodes = pd.DataFrame(rows_list, columns=column_names)

# Drop rows where 'Borough' == 'Not assigned'
indexNames = postalcodes[postalcodes['Borough'] == 'Not assigned'].index
postalcodes.drop(indexNames, inplace=True)
postalcodes = postalcodes.reset_index()
del postalcodes['index']

# Assign 'Borough' where 'Neighborhood' == Not assigned'
postalcodes.loc[postalcodes['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = postalcodes['Borough']

# Combine Neighborhoods in rows with the same PostalCode and Borough
postalcodes_group = postalcodes.groupby(['PostalCode', 'Borough'], sort=False).agg(lambda x: ', '.join(x))
postalcodes_group = postalcodes_group.reset_index()

# Show the first 5 rows from the new DataFrame
postalcodes_group.head()

number_of_rows = postalcodes_group.shape[0]
print('The number of rows is ', number_of_rows)


The number of rows is  103


## 2.- Get the latitude and the longitude coordinates of each neighborhood.

### Read the .csv file

In [8]:
coordinates_df = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Update "Postal Code" column name

In [9]:
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge data with data obtained from Part 1

In [10]:
merged_dataframe = pd.merge(postalcodes_group, coordinates_df, how='left', on='PostalCode')
merged_dataframe

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## 3.- Explore and cluster the neigborhoods in Toronto.

### Import libraries needed

In [18]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


### Obtain Toronto coordinates with geopy library

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with folium

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

### Filter data that includes the word Toronto

In [19]:
merged_dataframe_Toronto = merged_dataframe[merged_dataframe['Borough'].str.contains("Toronto")]
merged_dataframe_Toronto = merged_dataframe_Toronto.reset_index()
del merged_dataframe_Toronto['index']
dfToronto = merged_dataframe_Toronto
merged_dataframe_Toronto.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


### Create map of Toronto using latitude and longitude values

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


### Add markers to map


In [21]:
for lat, lng, postalcode, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['PostalCode'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}, {}'.format(postalcode, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### We play with the first neighbourhood to check all venues to perform the search and group function for all of them later

In [38]:
dfToronto.loc[0, 'Neighborhood']

neighborhood_latitude = dfToronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

## Top 50 venues within a radius of 500 m with my Foursquare account

CLIENT_ID = 'RYZSYSHJA4ZWVVWX24N51XNVW0ONA4PPRIZLX0JDAQ3UND2G' # your Foursquare ID
CLIENT_SECRET = 'D5ZO52ZZNRFDACCB1UR2JRXN224UT41PCUJ0A32QLFCPZJQJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50 # max venues to retrieve
radius = 500 # radius from Harbourfront, Regent Park's coordinates

# create URL and name ir "url" using the method "explore"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# Copy the result of the query in a dictionary
results = requests.get(url).json()
type(results)

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


dict

### Extraction of the category of the venue

In [41]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON


# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


### Same process to all neighborhoods in Toronto

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])


    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )



Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

### Report venues returned for each neighborhood

In [46]:
toronto_venues.groupby('Neighborhood').count().head(20)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",50,50,50,50,50,50
Business Reply Mail Processing Centre 969 Eastern,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",29,29,29,29,29,29
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50


In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",50,50,50,50,50,50
Business Reply Mail Processing Centre 969 Eastern,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",29,29,29,29,29,29
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50


In [52]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


### Create a new empty dataframe with all venues

In [55]:
# Drop rows where 'Venue Category' == 'Neighborhood'
indexNamesVC = toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'].index
toronto_venues.drop(indexNamesVC, inplace=True)
toronto_venues = toronto_venues.reset_index()
del toronto_venues['index']
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.040816,0.00,0.000000,0.000000,0.000000,0.020408,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.020408,0.040816,0.000000,0.000000,0.020408,0.020408,0.000000,0.00,0.00,0.061224,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,0.00,0.000000,0.000000,0.040816,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.020408,0.000000,0.00,0.000000

### We can see the 10 top venues for neighborhood

In [57]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0  American Restaurant  0.06
1           Steakhouse  0.06
2                 Café  0.06
3          Coffee Shop  0.06
4                Hotel  0.04
5       Breakfast Spot  0.04
6     Asian Restaurant  0.04
7         Concert Hall  0.04
8            Gastropub  0.04
9          Pizza Place  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.06
2            Creperie  0.04
3              Bakery  0.04
4  Seafood Restaurant  0.04
5        Cocktail Bar  0.04
6      Farmers Market  0.04
7         Cheese Shop  0.04
8               Hotel  0.04
9            Beer Bar  0.02


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.06
2              Restaurant  0.06
3  Furniture / Home Store  0.04
4                     Bar  0.04
5                  Bakery  0.04
6  Thrift / Vintage Store  0.04
7      

                   venue  freq
0                    Bar  0.10
1       Asian Restaurant  0.06
2            Coffee Shop  0.06
3            Pizza Place  0.04
4                   Café  0.04
5           Cocktail Bar  0.04
6  Vietnamese Restaurant  0.04
7             Restaurant  0.04
8            Men's Store  0.04
9                 Bakery  0.04


----Moore Park, Summerhill East----
                 venue  freq
0                 Park  0.15
1        Grocery Store  0.15
2           Playground  0.08
3                 Café  0.08
4          Candy Store  0.08
5       Sandwich Place  0.08
6                 Bank  0.08
7      Thai Restaurant  0.08
8  Japanese Restaurant  0.08
9         Tennis Court  0.08


----North Toronto West----
                 venue  freq
0  Sporting Goods Shop  0.09
1          Coffee Shop  0.09
2   Italian Restaurant  0.06
3                Diner  0.06
4         Skating Rink  0.06
5                 Café  0.06
6                  Spa  0.03
7          Flower Shop  0.03
8   Mexican 

### Loading this information in a *Pandas* dataframe

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # print('Indice')
    # print(row_categories_sorted[0:num_top_venues])
    # print('Valor')
    # print(row_categories_sorted.index[0:num_top_venues])
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Steakhouse,American Restaurant,Concert Hall,Asian Restaurant,Hotel,Pizza Place,Gastropub,Breakfast Spot
1,Berczy Park,Coffee Shop,Café,Farmers Market,Cheese Shop,Cocktail Bar,Hotel,Creperie,Bakery,Seafood Restaurant,Bistro
2,"Brockton, Exhibition Place, Parkdale Village",Café,Restaurant,Coffee Shop,Furniture / Home Store,Sandwich Place,Bar,Hotel,Thrift / Vintage Store,Bakery,Gift Shop
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Bar,Harbor / Marina,Brewery,Italian Restaurant,Pizza Place,Burrito Place,Grocery Store,Pub,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Boat or Ferry,Airport Terminal,Airport Lounge,Rental Car Location,Airport Service,Sculpture Garden,Tunnel,Plane,Boutique


### Cluster neighborhoods with k-means

In [67]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfToronto

# merge toronto_grouped with dfToronto (Toronto data) to add latitude/longitude for each neighborhood 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head()


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

#### Cluster 1

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,0,Coffee Shop,Bookstore,Park,Restaurant,Café,Bus Line,Gym / Fitness Center,Airport Service,Filipino Restaurant,Farmers Market


#### Cluster 2

In [70]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Park,Café,Bakery,Theater,Thai Restaurant,Mexican Restaurant,Pub,Breakfast Spot
1,Downtown Toronto,1,Clothing Store,Café,Coffee Shop,Italian Restaurant,Cosmetics Shop,Theater,Ramen Restaurant,Modern European Restaurant,Shopping Mall,Burger Joint
2,Downtown Toronto,1,Café,Restaurant,Gastropub,Coffee Shop,Farmers Market,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Park,Gym
3,East Toronto,1,Pub,Bar,Japanese Restaurant,Sandwich Place,Breakfast Spot,Café,Beach,Ice Cream Shop,Indian Restaurant,Indie Movie Theater
4,Downtown Toronto,1,Coffee Shop,Café,Farmers Market,Cheese Shop,Cocktail Bar,Hotel,Creperie,Bakery,Seafood Restaurant,Bistro
5,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Tea Room,Pizza Place,Yoga Studio,Plaza,Seafood Restaurant
6,Downtown Toronto,1,Korean Restaurant,Grocery Store,Coffee Shop,Café,Diner,Indian Restaurant,Cocktail Bar,Park,Pizza Place,Japanese Restaurant
7,Downtown Toronto,1,Café,Coffee Shop,Steakhouse,American Restaurant,Concert Hall,Asian Restaurant,Hotel,Pizza Place,Gastropub,Breakfast Spot
8,West Toronto,1,Coffee Shop,Park,Pharmacy,Portuguese Restaurant,Bakery,Café,Bar,Supermarket,Gym,Pool
9,Downtown Toronto,1,Park,Coffee Shop,Hotel,Baseball Stadium,Aquarium,Fried Chicken Joint,Café,Brewery,Japanese Restaurant,Beer Bar


#### Cluster 3

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Playground,Pet Store,Garden,Home Service,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant


#### Cluster 4 

In [72]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,3,Grocery Store,Park,Sandwich Place,Japanese Restaurant,Bank,Café,Tennis Court,Thai Restaurant,Candy Store,Playground
32,Downtown Toronto,3,Park,Bank,Trail,Candy Store,Grocery Store,Playground,Donut Shop,Discount Store,Dive Bar,Dog Run
